# Clase: Toma de decisiones con IA

Ejemplo simple de **modelo de clasificación** para mostrar en clases.

Escenario ficticio: *riesgo de abandono escolar* según asistencia, promedio de notas, reprobaciones y apoyo familiar.

## 1. Importar librerías

En Colab no hace falta instalar nada extra para este ejemplo.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


## 2. Crear un dataset ficticio

Simularemos 500 estudiantes con variables sencillas:
- `asistencia_pct`: porcentaje de asistencia (50 a 100).
- `promedio_nota`: nota promedio (de 2.0 a 7.0).
- `reprobaciones`: cantidad de ramos reprobados.
- `apoyo_familiar`: 1 si tiene apoyo familiar alto, 0 si es bajo.

La etiqueta será `riesgo_alto` (1 = alto riesgo de abandono, 0 = bajo).

In [ ]:
np.random.seed(42)

n = 500

asistencia = np.clip(np.random.normal(85, 10, n), 50, 100)
promedio = np.clip(np.random.normal(5, 1, n), 2, 7)
reprobaciones = np.random.poisson(0.5, n)
apoyo_familiar = np.random.binomial(1, 0.6, n)

# Regla simple para definir el riesgo (esto es *ficticio*)
puntaje_riesgo = (
    (asistencia < 80).astype(int) +
    (promedio < 4.5).astype(int) +
    (reprobaciones >= 1).astype(int) +
    (apoyo_familiar == 0).astype(int)
)

riesgo_alto = (puntaje_riesgo >= 2).astype(int)

df = pd.DataFrame({
    "asistencia_pct": asistencia,
    "promedio_nota": promedio,
    "reprobaciones": reprobaciones,
    "apoyo_familiar": apoyo_familiar,
    "riesgo_alto": riesgo_alto
})

df.head()


## 3. Separar variables de entrada y salida

- X: características de los estudiantes.
- y: si están en alto riesgo o no.

In [ ]:
X = df[["asistencia_pct", "promedio_nota", "reprobaciones", "apoyo_familiar"]]
y = df["riesgo_alto"]

X.shape, y.shape


## 4. Dividir en datos de entrenamiento y prueba

Usaremos 70% para entrenar el modelo y 30% para probar qué tan bien generaliza.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Tamaño entrenamiento:", X_train.shape[0])
print("Tamaño prueba:", X_test.shape[0])


## 5. Entrenar un modelo de clasificación

Usaremos **Regresión Logística**, un modelo muy clásico para problemas de clasificación binaria.

In [ ]:
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

print("Modelo entrenado 👍")        


## 6. Evaluar el modelo

Miraremos el **informe de clasificación** y la **matriz de confusión** para ver cómo se comporta.

In [ ]:
y_pred = modelo.predict(X_test)

print("\nInforme de clasificación:\n")
print(classification_report(y_test, y_pred))

print("Matriz de confusión:\n")
print(confusion_matrix(y_test, y_pred))


## 7. Probar con casos individuales

Ahora probamos con 1 o 2 estudiantes inventados, para mostrar cómo el modelo toma decisiones.

In [ ]:
# Caso 1: buena asistencia, buenas notas, sin reprobaciones, buen apoyo familiar
caso_bajo_riesgo = pd.DataFrame({
    "asistencia_pct": [93],
    "promedio_nota": [6.2],
    "reprobaciones": [0],
    "apoyo_familiar": [1]
})

# Caso 2: baja asistencia, malas notas, varias reprobaciones, poco apoyo familiar
caso_alto_riesgo = pd.DataFrame({
    "asistencia_pct": [70],
    "promedio_nota": [3.8],
    "reprobaciones": [2],
    "apoyo_familiar": [0]
})

print("Predicción caso 1 (0 = bajo riesgo, 1 = alto riesgo):")
print(modelo.predict(caso_bajo_riesgo))

print("\nPredicción caso 2 (0 = bajo riesgo, 1 = alto riesgo):")
print(modelo.predict(caso_alto_riesgo))


## 8. Resumen para la clase

- El modelo **aprende patrones** a partir de ejemplos (datos históricos).
- Luego, dado un caso nuevo, **clasifica** en `alto riesgo` o `bajo riesgo`.
- La decisión no es mágica: se basa en las variables que definimos (asistencia, notas, etc.).

Este mismo esquema se puede aplicar a:
- Riesgo de reingreso hospitalario.
- Priorización de casos sociales.
- Clasificación de reclamos municipales.
- Detección de fraude, etc.